In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from pandas.plotting import autocorrelation_plot

from data_import import *

In [9]:
oregon_data_dict = oregon_import(float_32=True)

file import: 100%|██████████| 3/3 [00:01<00:00,  2.55it/s]


In [3]:
# wa_df = oregon_data_dict['train_timeseries'].iloc[4:,:].reset_index(inplace=False, drop=True)
wa_df = oregon_data_dict['train_timeseries'].copy()
wa_df = wa_df[wa_df['fips']==41067]
wa_df.drop(columns=['fips'],inplace=True)
wa_df = wa_df.iloc[4:,:]
wa_df = wa_df.iloc[:-4,:]
wa_df['date'] = wa_df['date'].map(pd.Timestamp.timestamp)
wa_df.reset_index(inplace=True,drop=True)

In [4]:
wa_county_score_list = wa_df['score'].dropna().copy()

In [ ]:
result = adfuller(wa_county_score_list)
print('ADF Statistic:', result[0])
print('p-value:', result[1])

ADF Statistic: -5.508469614198743
p-value: 1.9969355206027656e-06


In [13]:
train_copy = oregon_data_dict['train_timeseries'].copy()
train_copy.dropna(subset=['score'], how='all', inplace=True)

In [15]:
oregon_score_list = wa_df['score'].dropna().copy()
result = adfuller(oregon_score_list)
print('ADF Statistic:', result[0])
print('p-value:', result[1])

ADF Statistic: -5.508469614198743
p-value: 1.9969355206027656e-06


In [ ]:
model = ARIMA(airline_data_diff, order=(1, 0, 1))
model_fit = model.fit()

In [ ]:


def parser(x):
	return pd.Datetime.strptime('190'+x, '%Y-%m')

series = pd.read_csv('./processed_data/oregon_train_timeseries.csv', header=0, parse_dates=[0], index_col=0, date_parser=parser)
autocorrelation_plot(series)
plt.show()

C:\Users\pears\AppData\Local\Temp\ipykernel_21744\304317978.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  series = pd.read_csv('./processed_data/oregon_train_timeseries.csv', header=0, parse_dates=[0], index_col=0)


In [10]:
only_scored_train = oregon_data_dict['train_timeseries'].copy()
only_scored_train.dropna(subset=['score'], how='all', inplace=True)

In [7]:
import warnings
from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error


In [11]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		y_hat = model_fit.forecast()[0]
		predictions.append(y_hat)
		history.append(test[t])
	# calculate out of sample error
	rmse = np.sqrt(mean_squared_error(test, predictions))
	return rmse

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	# dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					rmse = evaluate_arima_model(dataset, order)
					if rmse < best_score:
						best_score, best_cfg = rmse, order
					print('ARIMA%s RMSE=%.3f' % (order,rmse))
				except:
					continue
	print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))


# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(only_scored_train.values, p_values, d_values, q_values)

Best ARIMANone RMSE=inf
